In [39]:
! pip install -U opencv-contrib-python
! git clone https://github.com/naver/r2d2.git

In [79]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/task1/')

Mounted at /content/drive


In [80]:
from pathlib import Path
from course_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import cv2


In [81]:
DATASET_PATH = Path() / 'midv500_compressed'
assert DATASET_PATH.exists(), DATASET_PATH.absolute()

In [82]:
! cd midv500_compressed ; cd 50_xpo_id ; cd images ; ls 

50_xpo_id.png  CA  CS  HA  HS  KA  KS  PA  PS  TA  TS


всего 15050 фоток

In [85]:
# Собираем список пакетов (MidvPackage) 
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

(50, course_ocr_t1.data.MidvPackage)

## Classic CV baseline

In [49]:
import sys
r2d2_path = '/content/drive/My Drive/task1/r2d2/'
if r2d2_path not in sys.path:
    sys.path.append(r2d2_path)


In [25]:
! cd r2d2

In [64]:
import os, pdb
from PIL import Image
import numpy as np
import torch

from r2d2.tools import common
# from tools.dataloader import norm_RGB
from r2d2.nets.patchnet import *
import torchvision.transforms as tvf
RGB_mean = [0.485, 0.456, 0.406]
RGB_std  = [0.229, 0.224, 0.225]

norm_RGB = tvf.Compose([tvf.ToTensor(), tvf.Normalize(mean=RGB_mean, std=RGB_std)])

### Finetuning R2D2

In [ ]:
! cd r2d2; python train.py --save-path /content/drive/MyDrive/task1/r2d2/models/r2d2.pt --net 'Fast_Quad_L2Net_ConfCFS()' --pretrained /content/drive/MyDrive/task1/r2d2/models/faster2d2_WASF_N16.pt --epochs 10 --train-data D

In [65]:
def load_network(model_fn): 
    checkpoint = torch.load(model_fn)
    # print("\n>> Creating net = " + checkpoint['net']) 
    net = eval(checkpoint['net'])
    nb_of_weights = common.model_size(net)
    # print(f" ( Model size: {nb_of_weights/1000:.0f}K parameters )")

    # initialization
    weights = checkpoint['state_dict']
    net.load_state_dict({k.replace('module.',''):v for k,v in weights.items()})
    return net.eval()
class NonMaxSuppression (torch.nn.Module):
    def __init__(self, rel_thr=0.7, rep_thr=0.7):
        nn.Module.__init__(self)
        self.max_filter = torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.rel_thr = rel_thr
        self.rep_thr = rep_thr
    
    def forward(self, reliability, repeatability, **kw):
        assert len(reliability) == len(repeatability) == 1
        reliability, repeatability = reliability[0], repeatability[0]

        # local maxima
        maxima = (repeatability == self.max_filter(repeatability))

        # remove low peaks
        maxima *= (repeatability >= self.rep_thr)
        maxima *= (reliability   >= self.rel_thr)

        return maxima.nonzero().t()[2:4]
def extract_multiscale( net, img, detector, scale_f=2**0.25, 
                        min_scale=0.0, max_scale=1, 
                        min_size=256, max_size=1024, 
                        verbose=False):
    old_bm = torch.backends.cudnn.benchmark 
    torch.backends.cudnn.benchmark = False # speedup
    
    # extract keypoints at multiple scales
    B, three, H, W = img.shape
    assert B == 1 and three == 3, "should be a batch with a single RGB image"
    
    assert max_scale <= 1
    s = 1.0 # current scale factor
    
    X,Y,S,C,Q,D = [],[],[],[],[],[]
    while  s+0.001 >= max(min_scale, min_size / max(H,W)):
        if s-0.001 <= min(max_scale, max_size / max(H,W)):
            nh, nw = img.shape[2:]
            if verbose: print(f"extracting at scale x{s:.02f} = {nw:4d}x{nh:3d}")
            # extract descriptors
            with torch.no_grad():
                res = net(imgs=[img])
                
            # get output and reliability map
            descriptors = res['descriptors'][0]
            reliability = res['reliability'][0]
            repeatability = res['repeatability'][0]

            # normalize the reliability for nms
            # extract maxima and descs
            y,x = detector(**res) # nms
            c = reliability[0,0,y,x]
            q = repeatability[0,0,y,x]
            d = descriptors[0,:,y,x].t()
            n = d.shape[0]

            # accumulate multiple scales
            X.append(x.float() * W/nw)
            Y.append(y.float() * H/nh)
            S.append((32/s) * torch.ones(n, dtype=torch.float32, device=d.device))
            C.append(c)
            Q.append(q)
            D.append(d)
        s /= scale_f

        # down-scale the image for next iteration
        nh, nw = round(H*s), round(W*s)
        img = F.interpolate(img, (nh,nw), mode='bilinear', align_corners=False)

    # restore value
    torch.backends.cudnn.benchmark = old_bm

    Y = torch.cat(Y)
    X = torch.cat(X)
    S = torch.cat(S) # scale
    scores = torch.cat(C) * torch.cat(Q) # scores = reliability * repeatability
    XYS = torch.stack([X,Y,S], dim=-1)
    D = torch.cat(D)
    return XYS, D, scores

In [66]:
net = load_network('/content/drive/My Drive/task1/r2d2/models/r2d2.pt')
iscuda = common.torch_set_gpu(0)
if iscuda: net = net.cuda()
def R2D2(img_path, top_k=5000):

    detector = NonMaxSuppression(
        rel_thr = 0.7, 
        rep_thr = 0.7)

    img = Image.open(img_path).convert('RGB')
    W, H = img.size
    img = norm_RGB(img)[None] 
    if iscuda: img = img.cuda()
    
    # extract keypoints/descriptors for a single image
    xys, desc, scores = extract_multiscale(net, img, detector,
        scale_f   = 2**0.25, 
        min_scale = 0, 
        max_scale = 1,
        min_size  = 256, 
        max_size  = 1024, 
        verbose = False)

    xys = xys.cpu().numpy()
    desc = desc.cpu().numpy()
    scores = scores.cpu().numpy()
    idxs = scores.argsort()[-top_k or None:]

    imsize = (W,H),
    keypoints = xys[idxs], 
    descriptors = desc[idxs], 
    scores = scores[idxs]

    return keypoints, descriptors, scores
def R2D2_hard(img_path, top_k=5000):

    detector = NonMaxSuppression(
        rel_thr = 0.7, 
        rep_thr = 0.7)

    img = Image.open(img_path).convert('RGB')
    W, H = img.size
    img = norm_RGB(img)[None] 
    if iscuda: img = img.cuda()
    
    # extract keypoints/descriptors for a single image
    xys, desc, scores = extract_multiscale(net, img, detector,
        scale_f   = 2**0.25, 
        min_scale = 0.3, 
        max_scale = 1,
        min_size  = 0, 
        max_size  = 9999, 
        verbose = False)

    xys = xys.cpu().numpy()
    desc = desc.cpu().numpy()
    scores = scores.cpu().numpy()
    idxs = scores.argsort()[-top_k or None:]

    imsize = (W,H),
    keypoints = xys[idxs], 
    descriptors = desc[idxs], 
    scores = scores[idxs]

    return keypoints, descriptors, scores

Launching on GPUs 0


In [67]:
class R2D2Crop:
    """
    Baseline на основе поиска и сопоставления ключевых точек на шаблоне(ref) и запросе(query)
    Алгоритм:
        0. Найти ключевые точки(КТ) на ref
        1. Найти ключевые точки(КТ) на query
        2. Сопоставить КТ query и ref друг-другу
        3. На основе сопоставления найти гомографию из ref в query
        4. Отобразить 4 угла изображения ref в query - это искомый четырехугольник кропа
    """
    def __init__(self, ref_image):
        self.ref_image = ref_image
        
        # 0. Найти ключевые точки(КТ) на ref
        self.ref_kpts, self.ref_fts = self.compute_kps(
            self.ref_image
        )

    def compute_kps(self, img, response_quantile=0.25, count_kps=5000):
        img_path = '/content/drive/MyDrive/task1/' + str(img.img_path)

        kp, features, score = R2D2(img_path)

        return kp[0], features[0]
    
    def find_qr_homography(self, kp_q, ft_q, kp_r, ft_r, ratio=0.75):
        # 2. Сопоставить КТ query и ref друг-другу
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(ft_q, ft_r, k=2)
        good = []

        # Дополнителный опциональный шаг - фильтрация Lowe, Lowe ratio-test:
        # Пусть есть лучший матч1 (query-kp, ref-kp)
        # Если есть другой возможный матч2 (query-kp, ref-kp2), и их качество
        # (т.е. расстояние в пространстве features) маленькое,
        # то нельзя быть уверенном, что матч1 - надежный, и проще его выбросить
        for m,n in matches:
            if m.distance <  ratio * n.distance:
                good.append([m])
        # if len(good) < 4:
        #     good=[]
        #     # min_m = [m for m, n in matches]
        #     # good = sorted(min_m)[:4]
        #     for m,n in matches:
        #         good.append([m])

        ptsA = np.zeros((len(good), 2), dtype="float")
        ptsB = np.zeros((len(good), 2), dtype="float")

        for (i, mlist) in enumerate(good):
            m = mlist[0]
            ptsA[i] = kp_q[m.queryIdx][:2]
            ptsB[i] = kp_r[m.trainIdx][:2]
        # 3. На основе сопоставления найти гомографию из ref в query
        (H, mask) = cv2.findHomography(ptsB, ptsA, method=cv2.RANSAC)
        return H
              
    def compute_crop(self, ref_img, hmg):
        # 4. Отобразить 4 угла изображения ref в query - это искомый четырехугольник кропа
        h, w, _ = np.array(ref_img.image).shape
        h = float(h)
        w = float(w)
        points = [
            [0, 0],
            [w, 0],
            [w, h],
            [0, h],
        ]
        points_crop = []
        for p in points:
            # применяем гомографию к точке, представленной в гомогенных координатах
            vec = hmg @ np.array([p[0], p[1], 1])
            points_crop.append([vec[0]/vec[-1], vec[1]/vec[-1]])
        return np.array(points_crop)
        
    def __call__(self, query_image):
        # 1. Найти ключевые точки(КТ) на query
        kpts, fts = self.compute_kps(query_image)
        
        homography = self.find_qr_homography(
            kpts, fts, self.ref_kpts, self.ref_fts
        )
        abs_crop = self.compute_crop(self.ref_image, homography)
        h, w, _ =  np.array(query_image.image).shape
        rel_crop = abs_crop / np.array([[w, h]])
        return rel_crop

In [68]:
class R2D2Crop_hard:
    """
    Baseline на основе поиска и сопоставления ключевых точек на шаблоне(ref) и запросе(query)
    Алгоритм:
        0. Найти ключевые точки(КТ) на ref
        1. Найти ключевые точки(КТ) на query
        2. Сопоставить КТ query и ref друг-другу
        3. На основе сопоставления найти гомографию из ref в query
        4. Отобразить 4 угла изображения ref в query - это искомый четырехугольник кропа
    """
    def __init__(self, ref_image):
        self.ref_image = ref_image
        
        # 0. Найти ключевые точки(КТ) на ref
        self.ref_kpts, self.ref_fts = self.compute_kps(
            self.ref_image
        )

    def compute_kps(self, img, response_quantile=0.25, count_kps=5000):
        img_path = '/content/drive/MyDrive/task1/' + str(img.img_path)

        kp, features, score = R2D2_hard(img_path)

        return kp[0], features[0]
    
    def find_qr_homography(self, kp_q, ft_q, kp_r, ft_r, ratio=0.9):
        # 2. Сопоставить КТ query и ref друг-другу
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(ft_q, ft_r, k=2)
        good = []

        # Дополнителный опциональный шаг - фильтрация Lowe, Lowe ratio-test:
        # Пусть есть лучший матч1 (query-kp, ref-kp)
        # Если есть другой возможный матч2 (query-kp, ref-kp2), и их качество
        # (т.е. расстояние в пространстве features) маленькое,
        # то нельзя быть уверенном, что матч1 - надежный, и проще его выбросить
        for m,n in matches:
            if m.distance <  ratio * n.distance:
                good.append([m])
        if len(good) < 4:
            good=[]
            # min_m = [m for m, n in matches]
            # good = sorted(min_m)[:4]
            for m,n in matches:
                good.append([m])

        ptsA = np.zeros((len(good), 2), dtype="float")
        ptsB = np.zeros((len(good), 2), dtype="float")

        for (i, mlist) in enumerate(good):
            m = mlist[0]
            ptsA[i] = kp_q[m.queryIdx][:2]
            ptsB[i] = kp_r[m.trainIdx][:2]
        # 3. На основе сопоставления найти гомографию из ref в query
        (H, mask) = cv2.findHomography(ptsB, ptsA, method=cv2.RANSAC)
        return H
              
    def compute_crop(self, ref_img, hmg):
        # 4. Отобразить 4 угла изображения ref в query - это искомый четырехугольник кропа
        h, w, _ = np.array(ref_img.image).shape
        h = float(h)
        w = float(w)
        points = [
            [0, 0],
            [w, 0],
            [w, h],
            [0, h],
        ]
        points_crop = []
        for p in points:
            # применяем гомографию к точке, представленной в гомогенных координатах
            vec = hmg @ np.array([p[0], p[1], 1])
            points_crop.append([vec[0]/vec[-1], vec[1]/vec[-1]])
        return np.array(points_crop)
        
    def __call__(self, query_image):
        # 1. Найти ключевые точки(КТ) на query
        kpts, fts = self.compute_kps(query_image)
        
        homography = self.find_qr_homography(
            kpts, fts, self.ref_kpts, self.ref_fts
        )
        abs_crop = self.compute_crop(self.ref_image, homography)
        h, w, _ =  np.array(query_image.image).shape
        rel_crop = abs_crop / np.array([[w, h]])
        return rel_crop

In [69]:
class SimpleBaselineCrop:
    """
    Baseline на основе поиска и сопоставления ключевых точек на шаблоне(ref) и запросе(query)
    Алгоритм:
        0. Найти ключевые точки(КТ) на ref
        1. Найти ключевые точки(КТ) на query
        2. Сопоставить КТ query и ref друг-другу
        3. На основе сопоставления найти гомографию из ref в query
        4. Отобразить 4 угла изображения ref в query - это искомый четырехугольник кропа
    """
    def __init__(self, ref_image):
        self.ref_image = ref_image
        
        # 0. Найти ключевые точки(КТ) на ref
        self.ref_kpts, self.ref_fts = self.compute_kps(
            self.ref_image
        )

    def compute_kps(self, img, response_quantile=0.25):
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        sift = cv2.SIFT_create()
        kp = sift.detect(gray, None)
        # фильтруем 25% неуверенных точек
        q_val = np.quantile(list(k.response for k in kp), response_quantile)
        kp = [k for k in kp if k.response > q_val]
        
        kp, features = sift.compute(gray, kp)
        return kp, features 
    
    def find_qr_homography(self, kp_q, ft_q, kp_r, ft_r, ratio=0.75):
        # 2. Сопоставить КТ query и ref друг-другу
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(ft_q, ft_r, k=2)
        good = []

        # Дополнителный опциональный шаг - фильтрация Lowe, Lowe ratio-test:
        # Пусть есть лучший матч1 (query-kp, ref-kp)
        # Если есть другой возможный матч2 (query-kp, ref-kp2), и их качество
        # (т.е. расстояние в пространстве features) маленькое,
        # то нельзя быть уверенном, что матч1 - надежный, и проще его выбросить
        for m,n in matches:
            if m.distance <  ratio * n.distance:
                good.append([m])
                
        ptsA = np.zeros((len(good), 2), dtype="float")
        ptsB = np.zeros((len(good), 2), dtype="float")

        for (i, mlist) in enumerate(good):
            m = mlist[0]
            ptsA[i] = kp_q[m.queryIdx].pt
            ptsB[i] = kp_r[m.trainIdx].pt
        # 3. На основе сопоставления найти гомографию из ref в query
        (H, mask) = cv2.findHomography(ptsB, ptsA, method=cv2.RANSAC)
        return H
              
    def compute_crop(self, ref_img, hmg):
        # 4. Отобразить 4 угла изображения ref в query - это искомый четырехугольник кропа
        h, w, _ = ref_img.shape
        h = float(h)
        w = float(w)
        points = [
            [0, 0],
            [w, 0],
            [w, h],
            [0, h],
        ]
        points_crop = []
        for p in points:
            # применяем гомографию к точке, представленной в гомогенных координатах
            vec = hmg @ np.array([p[0], p[1], 1])
            points_crop.append([vec[0]/vec[-1], vec[1]/vec[-1]])
        return np.array(points_crop)
        
    def __call__(self, query_image):
        # 1. Найти ключевые точки(КТ) на query
        kpts, fts = self.compute_kps(query_image)
        
        homography = self.find_qr_homography(
            kpts, fts, self.ref_kpts, self.ref_fts
        )
        abs_crop = self.compute_crop(self.ref_image, homography)
        h, w, _ = query_image.shape
        rel_crop = abs_crop / np.array([[w, h]])
        return rel_crop

In [70]:
class SimpleBaselineCropEasy:
    """
    Baseline на основе поиска и сопоставления ключевых точек на шаблоне(ref) и запросе(query)
    Алгоритм:
        0. Найти ключевые точки(КТ) на ref
        1. Найти ключевые точки(КТ) на query
        2. Сопоставить КТ query и ref друг-другу
        3. На основе сопоставления найти гомографию из ref в query
        4. Отобразить 4 угла изображения ref в query - это искомый четырехугольник кропа
    """
    def __init__(self, ref_image):
        self.ref_image = ref_image
        
        # 0. Найти ключевые точки(КТ) на ref
        self.ref_kpts, self.ref_fts = self.compute_kps(
            self.ref_image
        )

    def compute_kps(self, img, response_quantile=0.25):
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        sift = cv2.SIFT_create()
        kp = sift.detect(gray, None)
        # фильтруем 25% неуверенных точек
        q_val = np.quantile(list(k.response for k in kp), response_quantile)
        kp = [k for k in kp if k.response > q_val]
        
        kp, features = sift.compute(gray, kp)
        return kp, features 
    
    def find_qr_homography(self, kp_q, ft_q, kp_r, ft_r, ratio=0.75):
        # 2. Сопоставить КТ query и ref друг-другу
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(ft_q, ft_r, k=2)
        good = []

        # Дополнителный опциональный шаг - фильтрация Lowe, Lowe ratio-test:
        # Пусть есть лучший матч1 (query-kp, ref-kp)
        # Если есть другой возможный матч2 (query-kp, ref-kp2), и их качество
        # (т.е. расстояние в пространстве features) маленькое,
        # то нельзя быть уверенном, что матч1 - надежный, и проще его выбросить
        for m,n in matches:
            good.append([m])
                
        ptsA = np.zeros((len(good), 2), dtype="float")
        ptsB = np.zeros((len(good), 2), dtype="float")

        for (i, mlist) in enumerate(good):
            m = mlist[0]
            ptsA[i] = kp_q[m.queryIdx].pt
            ptsB[i] = kp_r[m.trainIdx].pt
        # 3. На основе сопоставления найти гомографию из ref в query
        (H, mask) = cv2.findHomography(ptsB, ptsA, method=cv2.RANSAC)
        return H
              
    def compute_crop(self, ref_img, hmg):
        # 4. Отобразить 4 угла изображения ref в query - это искомый четырехугольник кропа
        h, w, _ = ref_img.shape
        h = float(h)
        w = float(w)
        points = [
            [0, 0],
            [w, 0],
            [w, h],
            [0, h],
        ]
        points_crop = []
        for p in points:
            # применяем гомографию к точке, представленной в гомогенных координатах
            vec = hmg @ np.array([p[0], p[1], 1])
            points_crop.append([vec[0]/vec[-1], vec[1]/vec[-1]])
        return np.array(points_crop)
        
    def __call__(self, query_image):
        # 1. Найти ключевые точки(КТ) на query
        kpts, fts = self.compute_kps(query_image)
        
        homography = self.find_qr_homography(
            kpts, fts, self.ref_kpts, self.ref_fts
        )
        abs_crop = self.compute_crop(self.ref_image, homography)
        h, w, _ = query_image.shape
        rel_crop = abs_crop / np.array([[w, h]])
        return rel_crop

### Посмотрим какую точность выдаст усиленный r2d2
Получилось 0.9184


In [86]:
from shapely import geometry
results_dict = dict()

for j in tqdm(range(len(data_packs)), desc='Directory', leave=True, position=0):
    dp = data_packs[j]

    hard = R2D2Crop_hard(dp.template_item)

    for i in tqdm(range(len(dp)), desc='Files', leave=False, position=1):

        if dp[i].is_test_split():          
            try:
                pred_quad =  hard(dp[i]) 
                try:
                    _ = geometry.Polygon(pred_quad)
                    results_dict[dp[i].unique_key] = pred_quad
                except:
                    pass
            except Exception as exc:
                pass

Directory:   0%|          | 0/50 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

...

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

In [87]:
from course_ocr_t1.metrics import dump_results_dict, measure_crop_accuracy

In [88]:
dump_results_dict(results_dict, Path() / 'pred.json')

In [89]:
acc = measure_crop_accuracy(
    Path() / 'pred.json',
    Path() / 'gt.json'
)

In [90]:
print("Точность кропа: {:1.4f}".format(acc))

Точность кропа: 0.9184


### Лучшее решение выше baseline 0.9621 > 0.9609
В сложных примерах используем r2d2, а по умолчанию baseline

In [97]:
results_dict = dict()

for j in tqdm(range(len(data_packs)), desc='Directory', leave=True, position=0):
    dp = data_packs[j]

    bcm = SimpleBaselineCrop(np.array(dp.template_item.image))
    hard = R2D2Crop_hard(dp.template_item)

    for i in tqdm(range(len(dp)), desc='Files', leave=False, position=1):

        if dp[i].is_test_split():

            try:
                pred_quad = bcm(np.array(dp[i].image))
                results_dict[dp[i].unique_key] = pred_quad

            except Exception as exc:
                pass                
                try:
                    pred_quad =  hard(dp[i]) 
                    try:
                        _ = geometry.Polygon(pred_quad)
                        results_dict[dp[i].unique_key] = pred_quad
                    except:
                        pass
                except Exception as exc:
                    pass

Directory:   0%|          | 0/50 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

...

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

Files:   0%|          | 0/300 [00:00<?, ?it/s]

In [99]:
dump_results_dict(results_dict, Path() / 'pred.json')

In [100]:
acc = measure_crop_accuracy(
    Path() / 'pred.json',
    Path() / 'gt.json'
)

In [101]:
print("Точность кропа: {:1.4f}".format(acc))

Точность кропа: 0.9621
